# Análise Integrada de Métricas do Scrapy

Este notebook coleta e analisa métricas do projeto **Scrapy** utilizando:
- **Radon**: complexidade ciclomática, LOC e comentários
- **jscpd**: duplicação de código
- **GitHub API**: tempo médio de resolução de issues
- **GitPython**: evolução de commits e contribuidores

Resultados devem ter sido previamente salvos em formato .json
analysis_data/
  radon_cc.json            (ou radon_cc.txt)
  radon_raw.json           (ou radon_raw.txt)
  radon_mi.json            (ou radon_mi.txt)
  jscpd-report/jscpd-report.json
  commits.json
  issues_closed.json
  contributors_last_year.jsonr.json



In [53]:
import json, os, math
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser as dateparse

DATA_DIR = Path("analysis_data")
assert DATA_DIR.exists(), "Crie a pasta analysis_data e coloque os arquivos extraídos conforme instruções."

def rank_from_cc(cc):
    # mapeamento radon-like
    if cc <= 5: return 'A'
    if cc <= 10: return 'B'
    if cc <= 20: return 'C'
    if cc <= 30: return 'D'
    if cc <= 40: return 'E'
    return 'F'

def safe_load_json(p):
    p = Path(p)
    if not p.exists():
        return None
    raw = open(p, "rb").read()
    for enc in ("utf-8-sig", "utf-16", "utf-16-le", "utf-16-be"):
        try:
            text = raw.decode(enc)
            return json.loads(text)
        except Exception:
            continue
    print(f"Falha ao carregar {p.name}, não consegui decodificar")
    return None




## 1. Carregar e analisar radon_cc.json 

complexidade por função

In [42]:
cc_json = safe_load_json(DATA_DIR/"radon_cc.json")
cc_df = None

if cc_json is None and (DATA_DIR/"radon_cc.txt").exists():
    # fallback: parse texto (caso não tenha JSON)
    txt = open(DATA_DIR/"radon_cc.txt", encoding='utf8').read().splitlines()
    rows = []
    # parsing simplificado - busca linhas com "file.py: <name> - <complexity>"
    for L in txt:
        L = L.strip()
        if L.startswith(" "): continue
        if ":" in L and "(" in L and ")" in L and " - " in L:
            # tentamos extrair arquivo quando houver
            # este parsing é heurístico; se o radon TXT for diferente, adapte
            pass
    print("Nenhum radon_cc.json encontrado e o parser TXT não foi implementado para seu formato. Se tiver radon_cc.txt cole aqui um trecho.")
else:
    # radon cc JSON normalmente tem { "path/file.py": [ { "name":..., "complexity": n, "lineno":...}, ... ], ...}
    records = []
    for f, items in cc_json.items():
        for it in items:
            rec = {
                "file": f,
                "name": it.get("name") or it.get("fullname") or it.get("type"),
                "complexity": it.get("complexity") or it.get("cc") or it.get("complexity_score"),
                "lineno": it.get("lineno")
            }
            records.append(rec)
    cc_df = pd.DataFrame(records)
    # estatísticas
    total_functions = len(cc_df)
    avg_cc = cc_df['complexity'].mean()
    dist = cc_df['complexity'].apply(rank_from_cc).value_counts().sort_index()
    print(f"Funções analisadas: {total_functions}")
    print(f"Complexidade média: {avg_cc:.2f}")
    print("Distribuição por rank:")
    print(dist)
    display(cc_df.sort_values('complexity', ascending=False).head(20))


Funções analisadas: 11119
Complexidade média: 2.82
Distribuição por rank:
complexity
A    9976
B     852
C     244
D      33
E       9
F       5
Name: count, dtype: int64


,file,name,complexity,lineno
10190,venv\Lib\site-packages\pip\_vendor\rich\table.py,_render,49,747
10108,venv\Lib\site-packages\pip\_vendor\rich\style.py,__init__,49,122
8029,venv\Lib\site-packages\pip\_vendor\msgpack\fal...,_pack,49,676
6267,venv\Lib\site-packages\pip\_internal\locations...,get_scheme,47,216
5874,venv\Lib\site-packages\pip\_internal\cli\autoc...,autocomplete,43,17
6066,venv\Lib\site-packages\pip\_internal\commands\...,run,39,281
5577,scrapy\tests\test_settings\__init__.py,test_get,36,247
10109,venv\Lib\site-packages\pip\_vendor\rich\style.py,__str__,35,285
10570,venv\Lib\site-packages\pip\_vendor\urllib3\con...,urlopen,35,535
10293,venv\Lib\site-packages\pip\_vendor\rich\traceb...,extract,34,413


## 2. Carregar radon_raw.json 

(LOC / LLOC / SLOC / comments) e calcular NCLOC e densidade de comentários

In [54]:
import pandas as pd

# Suponha que o JSON foi carregado e tem essa estrutura (uma lista de dicionários)
raw_json = [
    {"file": "analise_comits_contributors.py", "loc": 53, "lloc": 26, "sloc": 36, "comments": 11, "multi": 0, "blank": 7, "single_comments": 10},
    {"file": "radon_exporta_metricas.py", "loc": 38, "lloc": 25, "sloc": 25, "comments": 5, "multi": 0, "blank": 8, "single_comments": 5},
    {"file": "scrapy\\conftest.py", "loc": 128, "lloc": 68, "sloc": 94, "comments": 12, "multi": 0, "blank": 31, "single_comments": 3}
]

# A estrutura do JSON que você forneceu no prompt é, na verdade, um dicionário
# onde as chaves são os nomes dos arquivos. Se for essa a estrutura real,
# o seu código original está correto e o problema pode ser na função safe_load_json.
# Mas a mensagem de erro indica o contrário.
# Se o JSON for como o seu exemplo, a solução abaixo não funcionará.
# Vamos assumir o que o erro diz: raw_json é uma lista.

# Código corrigido para uma lista de dicionários
rows = []
for file_data in raw_json: # Itera sobre cada dicionário na lista
    rows.append({
        "file": file_data["file"],
        "loc": file_data.get("loc") or file_data.get("LOC"),
        "lloc": file_data.get("lloc") or file_data.get("LLOC"),
        "sloc": file_data.get("sloc") or file_data.get("SLOC"),
        "comments": file_data.get("comments") or file_data.get("Comments"),
        "multi": file_data.get("multi") or 0,
        "blank": file_data.get("blank") or 0
    })

# O restante do seu código permanece o mesmo
raw_df = pd.DataFrame(rows).astype({'loc':'int','lloc':'int','sloc':'int','comments':'int','blank':'int'})
total_lloc = raw_df['lloc'].sum()
total_sloc = raw_df['sloc'].sum()
total_comments = raw_df['comments'].sum()
comment_density = total_comments / total_sloc * 100 if total_sloc > 0 else float('nan')
print(f"Total LLOC: {total_lloc}, Total SLOC: {total_sloc}, Comentários: {total_comments}, % Comentários = {comment_density:.2f}%")

Total LLOC: 119, Total SLOC: 155, Comentários: 28, % Comentários = 18.06%


## 3. Índice de Manutenibilidade (radon_mi.json)jeto

In [44]:
mi_json = safe_load_json(DATA_DIR/"radon_mi.json")

if mi_json is None:
    print("radon_mi.json não encontrado ou inválido")
else:
    rows = []
    for file, metrics in mi_json.items():
        rows.append({
            "file": file,
            "mi": metrics.get("mi"),
            "rank": metrics.get("rank")
        })
    mi_df = pd.DataFrame(rows)
    avg_mi = mi_df["mi"].mean()
    print(f"Média geral do Índice de Manutenibilidade: {avg_mi:.2f}")
    print("\nArquivos com pior manutenibilidade:")
    display(mi_df.sort_values("mi").head(10))


Média geral do Índice de Manutenibilidade: 66.60

Arquivos com pior manutenibilidade:


,file,mi,rank
239,scrapy\tests\test_http_response.py,0.000000,C
391,scrapy\tests\test_settings\__init__.py,0.000000,C
620,venv\Lib\site-packages\pip\_vendor\pkg_resourc...,0.000000,C
234,scrapy\tests\test_feedexport.py,0.000000,C
587,venv\Lib\site-packages\pip\_vendor\distlib\uti...,0.000000,C
692,venv\Lib\site-packages\pip\_vendor\rich\consol...,0.000000,C
199,scrapy\tests\test_crawl.py,0.000000,C
238,scrapy\tests\test_http_request.py,0.000000,C
200,scrapy\tests\test_crawler.py,0.000000,C
225,scrapy\tests\test_engine.py,3.727745,C


## 4. Duplicação (jscpd-report)

In [45]:
jscpd = safe_load_json(DATA_DIR/"jscpd-report"/"jscpd-report.json")
if jscpd is None:
    print("Não achei jscpd report em analysis_data/jscpd-report/jscpd-report.json")
else:
    # vários formatos possíveis; tentamos extrair estatísticas comuns
    stats = jscpd.get('statistics') or jscpd.get('summary') or {}
    total_lines = stats.get('totalLines') or stats.get('total', {}).get('lines') or None
    total_dup_lines = stats.get('totalDuplicates') or stats.get('duplicates', {}).get('lines') or None
    if total_lines and total_dup_lines:
        dup_percent = total_dup_lines / total_lines * 100
        print(f"Total lines: {total_lines}, duplicated lines: {total_dup_lines}, % duplication = {dup_percent:.2f}%")
    else:
        # fallback: compute sum of duplicate blocks
        duplicates = jscpd.get('duplicates') or []
        total_dup = sum(d.get('lines',0) for d in duplicates)
        # precisamos do total de SLOC (usamos valor do raw_df)
        if 'total_sloc' in locals():
            dup_percent = total_dup / total_sloc * 100
            print(f"Duplicated lines (sum blocks) = {total_dup}, % = {dup_percent:.2f}% (usando total_sloc do radon)")
        else:
            print("Dados incompletos no report jscpd; visualize analysis_data/jscpd-report/jscpd-report.json para confirmar formato.")
    # Top arquivos com maior duplicação (heurística)
    if 'duplicates' in jscpd:
        file_counts = {}
        for d in jscpd['duplicates']:
            for f in d.get('sources', []):
                file_counts[f['name']] = file_counts.get(f['name'], 0) + d.get('lines', 0)
        top = sorted(file_counts.items(), key=lambda x: x[1], reverse=True)[:20]
        print("Top arquivos com maior duplicação (heurística):")
        display(pd.DataFrame(top, columns=['file','duplicated_lines']).head(10))


Duplicated lines (sum blocks) = 1342, % = 865.81% (usando total_sloc do radon)
Top arquivos com maior duplicação (heurística):


,file,duplicated_lines


## 5. Célula 6 — Commits por mês (commits.json)

In [47]:
import pandas as pd

# Supondo que commits.json foi carregado corretamente
commits = safe_load_json(DATA_DIR/"commits.json")
dfc = pd.DataFrame(commits)

# Força a conversão para datetime com fuso horário (utc=True)
# Isso resolve o erro de 'mixed time zones' e garante o tipo datetime.
dfc['date'] = pd.to_datetime(dfc['date'], utc=True, errors='coerce')

# Remove as linhas onde a conversão falhou (se houver alguma)
dfc.dropna(subset=['date'], inplace=True)

# Se ainda houver dados, prossiga com a análise
if not dfc.empty:
    # Agora a coluna 'date' é do tipo datetime, então .dt funciona
    dfc['month'] = dfc['date'].dt.to_period('M')
    
    # Restante do código, que agora deve funcionar
    by_month = dfc.groupby('month').size().rename('commits').reset_index()
    by_month['month'] = by_month['month'].astype(str)
    
    print("Últimos 12 meses (commits por mês):")
    display(by_month.tail(12))
    print("Média commits/mês:", by_month['commits'].mean())
else:
    print("Nenhum dado de commits válido encontrado após a conversão de datas.")

Últimos 12 meses (commits por mês):


/tmp/ipykernel_16644/4098352641.py:17: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  dfc['month'] = dfc['date'].dt.to_period('M')


,month,commits
196,2024-10,33
197,2024-11,37
198,2024-12,29
199,2025-01,43
200,2025-02,27
201,2025-03,158
202,2025-04,14
203,2025-05,60
204,2025-06,58
205,2025-07,19


Média commits/mês: 57.875


## 6. Tempo médio de fechamento de issues (issues_closed.json)

In [48]:
issues = safe_load_json(DATA_DIR/"issues_closed.json")
if issues is None:
    print("Arquivo issues_closed.json não encontrado. Veja instruções de extração.")
else:
    df_iss = pd.json_normalize(issues)
    # garante colunas existam
    if 'created_at' in df_iss.columns and 'closed_at' in df_iss.columns:
        df_iss['created_at'] = pd.to_datetime(df_iss['created_at'])
        df_iss['closed_at'] = pd.to_datetime(df_iss['closed_at'])
        df_iss = df_iss[df_iss['closed_at'].notna()]
        df_iss['days_to_close'] = (df_iss['closed_at'] - df_iss['created_at']).dt.total_seconds()/86400
        print("Estatísticas de tempo para fechar issues (dias):")
        display(df_iss['days_to_close'].describe())
        print("Média dias para fechar:", df_iss['days_to_close'].mean())
    else:
        print("Formato inesperado em issues_closed.json; verifique campos created_at e closed_at.")


Estatísticas de tempo para fechar issues (dias):


count    2721.000000
mean      243.102835
std       582.007651
min         0.000081
25%         0.451516
50%         7.204641
75%       138.591366
max      4199.540324
Name: days_to_close, dtype: float64

Média dias para fechar: 243.10283521853347


## 7. Contribuidores últimos 12 meses

In [49]:
cont = safe_load_json(DATA_DIR/"contributors_last_year.json")
if cont is None:
    print("contributors_last_year.json não encontrado.")
else:
    dfc = pd.DataFrame(cont)
    print("Número de contribuidores ativos (último ano):", len(dfc))
    display(dfc.sort_values('commits', ascending=False).head(20))


Número de contribuidores ativos (último ano): 38


,commits,author,email
0,253,Andrey Rakhmatullin,wrar@wrar.name
1,43,Adri├ín Chaves,adrian@chaves.io
2,7,Laerte Pereira,5853172+Laerte@users.noreply.github.com
3,5,Mikhail Korobov,kmike84@gmail.com
4,4,Rotzbua,Rotzbua@users.noreply.github.com
5,4,Thalison Fernandes,thalisondev@gmail.com
6,4,anubhav,protokoul@users.noreply.github.com
7,3,Laerte Pereira,laertefbk@gmail.com
8,3,M Ikram Ullah Khan,44160462+IkramKhanNiazi@users.noreply.github.com
9,3,Mehraz Hossain Rumman,59512321+MehrazRumman@users.noreply.github.com


## 8. Comparar resultados com suas metas (gera tabela 'Atende?')

In [52]:
# Ajuste estes thresholds conforme suas metas
metas = {
    "complexidade_media": 10,
    "densidade_comentarios_pct": 20,
    "duplicacao_pct": 5,
    "mi_min": 70,
    "commits_por_mes_min": 10,
    "dias_fechamento_max": 14,
    "contribuidores_min": 10
}

results = {}
# complexidade média (da célula 2)
results['complexidade_media'] = float(avg_cc) if 'avg_cc' in locals() else (cc_df['complexity'].mean() if cc_df is not None else None)
# densidade comentarios:
results['densidade_comentarios_pct'] = float(comment_density) if 'comment_density' in locals() else None
# duplicacao
results['duplicacao_pct'] = float(dup_percent) if 'dup_percent' in locals() else None
# MI
results['mi_media'] = float(avg_mi) if 'avg_mi' in locals() else (mi_df['mi'].mean() if 'mi_df' in locals() and not mi_df.empty else None)
# commits por mes (media)
results['commits_por_mes'] = float(by_month['commits'].mean()) if 'by_month' in locals() else None
# tempo fechamento
results['dias_fechamento'] = float(df_iss['days_to_close'].mean()) if 'df_iss' in locals() and 'days_to_close' in df_iss.columns else None
# contribuidores
results['contribuidores'] = int(len(dfc)) if 'dfc' in locals() else None

comp = []
comp.append(["Complexidade média", results['complexidade_media'], f"≤ {metas['complexidade_media']}", results['complexidade_media'] <= metas['complexidade_media'] if results['complexidade_media'] is not None else None])
comp.append(["Densidade comentários (%)", results['densidade_comentarios_pct'], f"≥ {metas['densidade_comentarios_pct']}%", results['densidade_comentarios_pct'] >= metas['densidade_comentarios_pct'] if results['densidade_comentarios_pct'] is not None else None])
comp.append(["Duplicação (%)", results['duplicacao_pct'], f"≤ {metas['duplicacao_pct']}%", results['duplicacao_pct'] <= metas['duplicacao_pct'] if results['duplicacao_pct'] is not None else None])
comp.append(["Índice Manutenibilidade (média)", results['mi_media'], f"≥ {metas['mi_min']}", results['mi_media'] >= metas['mi_min'] if results['mi_media'] is not None else None])
comp.append(["Commits / mês (média)", results['commits_por_mes'], f"≥ {metas['commits_por_mes_min']}", results['commits_por_mes'] >= metas['commits_por_mes_min'] if results['commits_por_mes'] is not None else None])
comp.append(["Tempo médio fechar issues (dias)", results['dias_fechamento'], f"≤ {metas['dias_fechamento_max']}", results['dias_fechamento'] <= metas['dias_fechamento_max'] if results['dias_fechamento'] is not None else None])
comp.append(["Contribuidores (últ. 1 ano)", results['contribuidores'], f"≥ {metas['contribuidores_min']}", results['contribuidores'] >= metas['contribuidores_min'] if results['contribuidores'] is not None else None])

comp_df = pd.DataFrame(comp, columns=['Questão / Métrica','Valor obtido','Meta','Atende?'])
display(comp_df)


,Questão / Métrica,Valor obtido,Meta,Atende?
0,Complexidade média,2.821567,≤ 10,True
1,Densidade comentários (%),18.064516,≥ 20%,False
2,Duplicação (%),865.806452,≤ 5%,False
3,Índice Manutenibilidade (média),66.603809,≥ 70,False
4,Commits / mês (média),57.875000,≥ 10,True
5,Tempo médio fechar issues (dias),243.102835,≤ 14,False
6,Contribuidores (últ. 1 ano),38.000000,≥ 10,True


## 9. Observações e exportar tabela final (CSV/JSON) para relatório

In [51]:
# salva o resumo para anexar no relatório
comp_df.to_csv(DATA_DIR/"metrics_summary.csv", index=False)
comp_df.to_json(DATA_DIR/"metrics_summary.json", orient='records', force_ascii=False)
print("Resumo salvo em analysis_data/metrics_summary.csv e .json")


Resumo salvo em analysis_data/metrics_summary.csv e .json
